In [106]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [107]:
df = pd.read_csv('gurgaon_properties_post_feature_selection_v2.csv')

In [108]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,study room,servant room,store room,furnishing_type,floor_category
0,house,sector 109,8.75,5.0,5.0,3+,New Property,3600.00,1.0,1.0,1.0,2.0,Medium floor
1,house,sector 48,6.40,4.0,4.0,3,Moderately Old,2700.00,0.0,1.0,1.0,2.0,Medium floor
2,flats,sector 49,2.90,4.0,4.0,3+,Moderately Old,2383.00,0.0,1.0,0.0,0.0,Medium floor
3,flats,manesar,0.90,3.0,3.0,3+,Moderately Old,2089.00,0.0,1.0,0.0,0.0,Medium floor
4,flats,sector 112,3.40,3.0,3.0,2,Relatively New,2225.56,0.0,0.0,0.0,2.0,High floor


In [109]:
df['furnishing_type'].value_counts()

furnishing_type
0.0    2371
2.0     994
1.0     185
Name: count, dtype: int64

In [110]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [111]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,study room,servant room,store room,furnishing_type,floor_category
0,house,sector 109,8.75,5.0,5.0,3+,New Property,3600.00,1.0,1.0,1.0,furnished,Medium floor
1,house,sector 48,6.40,4.0,4.0,3,Moderately Old,2700.00,0.0,1.0,1.0,furnished,Medium floor
2,flats,sector 49,2.90,4.0,4.0,3+,Moderately Old,2383.00,0.0,1.0,0.0,unfurnished,Medium floor
3,flats,manesar,0.90,3.0,3.0,3+,Moderately Old,2089.00,0.0,1.0,0.0,unfurnished,Medium floor
4,flats,sector 112,3.40,3.0,3.0,2,Relatively New,2225.56,0.0,0.0,0.0,furnished,High floor


In [112]:
X = df.drop(columns=['price'])
y = df['price']

In [113]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

# Ordinal Encoding

In [114]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'floor_category']

In [115]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), columns_to_encode)
    ])

In [116]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [117]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [118]:
scores.mean(),scores.std()

(0.842899072800518, 0.03364002838939808)

# OneHotEncoding¶

In [119]:
columns_to_encode = ['property_type', 'balcony','floor_category', 'furnishing_type']

In [120]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(handle_unknown='ignore'),['sector','agePossession','furnishing_type'])
        # ('cat1',OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

In [121]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [122]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [123]:
scores.mean()

0.8417734330848055

In [124]:
scores.std()

0.03497611855086834

In [125]:
scores.std()

0.03497611855086834

# Target Encoder

In [130]:
# import category_encoders as ce

# columns_to_encode = ['property_type','balcony', 'furnishing_type','floor_category']

# # Creating a column transformer for preprocessing
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area','study room', 'servant room', 'store room']),
#         ('cat', OrdinalEncoder(), columns_to_encode),
#         ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
#         ('target_enc', ce.TargetEncoder(), ['sector'])
#     ], 
    
#     remainder='passthrough'
# )

# Ensure categorical columns are of the correct type
#X['property_type'] = X['property_type'].astype('category')
#X['balcony'] = X['balcony'].astype('category')
#X['furnishing_type'] = X['furnishing_type'].astype('category')
#X['floor_category'] = X['floor_category'].astype('category')
# X['sector'] = X['sector'].astype('category')
# X['agePossession'] = X['agePossession'].astype('category')

# Columns to encode
columns_to_encode = ['property_type', 'balcony', 'furnishing_type', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'study room', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1', OneHotEncoder(drop='first', sparse_output=False), ['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)


In [131]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [132]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [133]:
print(scores.mean(), scores.std())

0.8128344086820409 0.03322209110406818


In [136]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [137]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [138]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [139]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [140]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [141]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.882894,0.526107
6,extra trees,0.881622,0.528764
5,random forest,0.884545,0.531835
7,gradient boosting,0.872119,0.568962
1,svr,0.844624,0.658498
9,mlp,0.837812,0.678814
4,decision tree,0.787234,0.724380
2,ridge,0.812865,0.731533
0,linear_reg,0.812834,0.731756
8,adaboost,0.801531,0.738547
